In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP
from statsmodels.stats.multitest import multipletests

from matplotlib.colors import LinearSegmentedColormap

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  


In [ ]:
outdir_fig = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures"

# load gene annotation and data

In [ ]:
gene_modules = GeneModules()
g, gs, ms = gene_modules.check_genes('Cdh13')
print("\t".join(g))
print("\t".join(gs))
print("\t".join(ms))

In [ ]:
# use those 286 genes
df = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/cheng21_cell_scrna/res/L23-ABC-genes-n288-n286unq-annot_v3_july8_2024.csv")
genes_l23 = df['gene'].astype(str).values
genes_l23a = df[df['P17on']=='A']['gene'].astype(str).values
genes_l23b = df[df['P17on']=='B']['gene'].astype(str).values
genes_l23c = df[df['P17on']=='C']['gene'].astype(str).values

print(genes_l23a.shape, genes_l23b.shape, genes_l23c.shape)
genes_grp = df['P17on'].astype(str).values
assert len(genes_l23) == len(np.unique(genes_l23))

genes_l23.shape

In [ ]:
scores_abc = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/scores_l23abc.csv", 
                         index_col=0,
                        )
scores_abc['scores_c-a'] = scores_abc['scores_c'] - scores_abc['scores_a']
scores_abc

In [ ]:
adata = anndata.read("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad")
adata

In [ ]:
adata.X = adata.raw.X

In [ ]:
adata.obs['scores_a'] = scores_abc.loc[adata.obs.index,'scores_a'].copy()
adata.obs['scores_b'] = scores_abc.loc[adata.obs.index,'scores_b'].copy()
adata.obs['scores_c'] = scores_abc.loc[adata.obs.index,'scores_c'].copy()
adata.obs['scores_c-a'] = scores_abc.loc[adata.obs.index,'scores_c-a'].copy()

In [ ]:
sample_labels = adata.obs['Sample'].values
time_labels = [s[:-1].replace('DR', '') for s in sample_labels]

adata.obs['sample'] = sample_labels #
adata.obs['time']   = time_labels

uniq_samples = natsorted(np.unique(sample_labels))
nr_samples = [s for s in uniq_samples if "DR" not in s]
dr_samples = [s for s in uniq_samples if "DR" in s]

uniq_conds = np.array(natsorted(np.unique(adata.obs['cond'].values)))
print(uniq_conds)

In [ ]:
# remove mitocondria genes
adata = adata[:,~adata.var.index.str.contains(r'^mt-')]

# filter genes
cond = np.ravel((adata.X>0).sum(axis=0)) > 10 # expressed in more than 10 cells
adata = adata[:,cond].copy()

In [ ]:
# counts
x = adata.X
cov = np.ravel(np.sum(x, axis=1))
genes = adata.var.index.values

# CP10k
xn = (sparse.diags(1/cov).dot(x))*1e4

# log2(CP10k+1)
xln = xn.copy()
xln.data = np.log2(xln.data+1)

adata.layers[    'norm'] = np.array(xn.todense())
adata.layers[ 'lognorm'] = np.array(xln.todense())

In [ ]:
# offset = 1
num_archetypal_cells = 100
SHUFFLE = False

n_cond = len(uniq_conds)
n_gene = adata.shape[1] 
qs_tensor   = np.zeros((n_cond,3,n_gene))  # 3 represents 3 pairwise comparisons (ca, ba, bc)
l2fc_tensor = np.zeros((n_cond,3,n_gene))

for cond_code, cond in enumerate(uniq_conds):
    # get sub
    adatasub = adata[adata.obs['cond']==cond]
    
    # get A vs C 
    ranks_ac = adatasub.obs['scores_c-a'].rank()
    ranks_b  = adatasub.obs['scores_b'].rank()
    
    precond_a = ranks_ac <= num_archetypal_cells
    precond_c = ranks_ac > adatasub.shape[0] - num_archetypal_cells
    precond_b = ranks_b  > adatasub.shape[0] - num_archetypal_cells
    
    cond_a = np.all([ precond_a, ~precond_b, ~precond_c], axis=0)
    cond_b = np.all([~precond_a,  precond_b, ~precond_c], axis=0)
    cond_c = np.all([~precond_a, ~precond_b,  precond_c], axis=0)
    
    print(precond_a.sum(), 
          precond_b.sum(), 
          precond_c.sum(),)
    print(cond_a.sum(), cond_b.sum(), cond_c.sum())
    
    adatasub_a = adatasub[cond_a]
    adatasub_b = adatasub[cond_b]
    adatasub_c = adatasub[cond_c]
    
    # DEGs
    mat_a = adatasub_a.layers['lognorm'][...]
    mat_b = adatasub_b.layers['lognorm'][...]
    mat_c = adatasub_c.layers['lognorm'][...]
    
    ts_ca, ps_ca = stats.ttest_ind(mat_c, mat_a)
    ts_ba, ps_ba = stats.ttest_ind(mat_b, mat_a)
    ts_bc, ps_bc = stats.ttest_ind(mat_b, mat_c)
    
    _, qs_ca, _, _ = multipletests(np.nan_to_num(ps_ca, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    _, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    _, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    
    lfc_ca = np.mean(mat_c, axis=0) - np.mean(mat_a, axis=0) # log2FC (log2CP10k as raw counts)
    lfc_ba = np.mean(mat_b, axis=0) - np.mean(mat_a, axis=0) # log2FC (log2CP10k as raw counts)
    lfc_bc = np.mean(mat_b, axis=0) - np.mean(mat_c, axis=0) # log2FC (log2CP10k as raw counts)
    
    num_sig_ca = np.sum(np.logical_and(qs_ca < 0.05, np.abs(lfc_ca) > 1))
    num_sig_b  = np.sum(np.all([
        qs_ba < 0.05, np.abs(lfc_ba) > 1, 
        qs_bc < 0.05, np.abs(lfc_bc) > 1, 
        ]))
    print(cond, num_sig_ca, num_sig_b)
    
    # save this
    l2fc_tensor[cond_code, 0] = lfc_ca
    l2fc_tensor[cond_code, 1] = lfc_ba
    l2fc_tensor[cond_code, 2] = lfc_bc
    
    qs_tensor[cond_code, 0] = qs_ca
    qs_tensor[cond_code, 1] = qs_ba
    qs_tensor[cond_code, 2] = qs_bc
    


In [ ]:
%%time
fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_qs_250408.npy'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_l2fc_250408.npy'
fout3 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_gene_list_250408.csv'

np.save(fout1, qs_tensor)
np.save(fout2, l2fc_tensor)

# check results and stats

In [ ]:
qs_tensor = np.load(fout1)
l2fc_tensor = np.load(fout2)

l2fc_th = np.log2(2)
alpha_th = 0.05

In [ ]:
qs   = qs_tensor[:,0,:]
l2fc = l2fc_tensor[:,0,:]

cond_sig_a = np.all([qs < alpha_th, l2fc < -l2fc_th], axis=0)
cond_sig_c = np.all([qs < alpha_th, l2fc >  l2fc_th], axis=0)

instances, counts_a = np.unique(cond_sig_a.sum(axis=0), return_counts=True)
instances, counts_c = np.unique(cond_sig_c.sum(axis=0), return_counts=True)

print('num A > C genes for each cond:\t', cond_sig_a.sum(axis=1), np.any(cond_sig_a, axis=0).sum())
print('num C > A genes for each cond:\t', cond_sig_c.sum(axis=1), np.any(cond_sig_c, axis=0).sum())
print('num A > C genes in num conds:\t',  counts_a[1:])
print('num C > A genes in num conds:\t',  counts_c[1:])

In [ ]:
qs_ba   = qs_tensor[:,1,:]
qs_bc   = qs_tensor[:,2,:]

l2fc_ba = l2fc_tensor[:,1,:]
l2fc_bc = l2fc_tensor[:,2,:]

qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two
l2fc_b = np.mean([l2fc_ba, l2fc_bc], axis=0) # mean fold change

cond_sig_b = np.all([l2fc_ba > 0, l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th], axis=0)
instances, counts_b = np.unique(cond_sig_b.sum(axis=0), return_counts=True)

print('num B > A,C per cond:\t',    cond_sig_b.sum(axis=1))
print('num B > A,C DARs in num conds:\t',  counts_b[1:])

In [ ]:
a_any = np.sort(adata.var[np.any(cond_sig_a, axis=0)].index.values)
c_any = np.sort(adata.var[np.any(cond_sig_c, axis=0)].index.values)
a_all = np.sort(adata.var[np.all(cond_sig_a, axis=0)].index.values)
c_all = np.sort(adata.var[np.all(cond_sig_c, axis=0)].index.values)
ac_overlap = np.sort(adata.var[np.logical_and(np.any(cond_sig_a, axis=0), np.any(cond_sig_c, axis=0))].index.values)

print('a any', a_any)
print('c any', c_any)
print('a all', a_all)
print('c all', c_all)
print('ac overlap', ac_overlap)

In [ ]:
a_all_annots, a_all_styled, a_all_annots_styled = gene_modules.check_genes(a_all)
c_all_annots, c_all_styled, c_all_annots_styled = gene_modules.check_genes(c_all)

print("\t".join(a_all_annots_styled)) # _styled))
print("---"*10) # _styled))
print("\t".join(c_all_annots_styled)) # _styled))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(np.arange(1,1+11)[::-1], np.cumsum(counts_a[1:][::-1]), '-o', label='A', color='C0')
ax.plot(np.arange(1,1+11)[::-1], np.cumsum(counts_c[1:][::-1]), '-o', label='C', color='C2')
ax.set_ylim(ymin=0)
ax.legend()
ax.set_ylabel('number of TFs')
ax.set_xlabel('number of time points')
sns.despine(ax=ax)
plt.show()

In [ ]:
df_res_all = []

for label, cond_sig in zip(['A', 'C', 'B'], 
                           [cond_sig_a, cond_sig_c, cond_sig_b]):
    
    cond_idx, gene_idx = np.nonzero(cond_sig.astype(int))
    
    df_res = pd.DataFrame()
    df_res['cond'] = uniq_conds[cond_idx]
    df_res['gene'] = genes[gene_idx]
    df_res['archetype'] = label
    df_res_all.append(df_res)
    
df_res_all = pd.concat(df_res_all)
df_res_all
    

In [ ]:
df_res_all.to_csv(fout3, header=True, index=False)

In [ ]:
df_res_all.groupby('gene').sum()

# Volcano

In [ ]:
def show_volcano_v2(thetypeidx, thetype, lfc, qs,
                    cond1, cond2up, cond2dn, 
                    querygenes_idx=None, 
                    gene_annots=None,
                    ax=None, bbox_to_anchor=(1,1), loc=None,
                   ): 
    """
    """
    eff = lfc[:,thetypeidx]
    pvl = -np.log10(np.clip(qs[:,thetypeidx], 1e-50, None)) # +1e-10)
    cnd_up = np.all([cond1[:,thetypeidx], 
                     cond2up[:,thetypeidx]], axis=0) 
    cnd_dn = np.all([cond1[:,thetypeidx], 
                     cond2dn[:,thetypeidx]], axis=0) 

    if ax is None: 
        fig, ax = plt.subplots()
    
    # all genes
    ax.scatter(eff, pvl, s=1, color='lightgray', rasterized=True)

    # up genes
    ax.scatter(eff[cnd_up], pvl[cnd_up], s=3, facecolors='C0', rasterized=True)
    # dn genes
    ax.scatter(eff[cnd_dn], pvl[cnd_dn], s=3, facecolors='C1', rasterized=True)
    
    # # query genes
    # add text
    if querygenes_idx is not None:
        ax.scatter(eff[querygenes_idx], pvl[querygenes_idx], s=15, 
                   # label=f'type-specific (n={len(querygenes_idx):,})',
                   facecolors='none', edgecolors='k', linewidth=1, rasterized=True)
        for idx in querygenes_idx:
            ax.text(eff[idx], pvl[idx], gene_annots[idx], fontsize=10)

            
    # ax.grid(axis='y')
    sns.despine(ax=ax)
    ax.set_xlabel('log2(FC) (C/A in CP10k)')
    ax.set_ylabel('-log10(adj. p)')
    ax.set_title(f'{thetype}')
    ax.text(1,0.1,
            f'up (n={cnd_up.sum():,})\ndown (n={cnd_dn.sum():,})', 
            ha='right',
            fontsize=10, transform=ax.transAxes)
    return 

In [ ]:
genes_comm = adata.var.index.values
lfc = l2fc_tensor[:,0,:].T
qs  = qs_tensor[:,0,:].T
lfc_th, qs_th = 1, 0.05

In [ ]:
cond1   = qs  <  qs_th
cond2up = lfc >  lfc_th
cond2dn = lfc < -lfc_th

In [ ]:
unq_conds = natsorted(np.unique(adata.obs['cond']))
unq_conds

In [ ]:
n = len(unq_conds)
fig, axs = plt.subplots(1,n,figsize=(4*n,4*1), sharex=True, sharey=True)
for cond_idx, thecond in enumerate(unq_conds):
    ax = axs.flat[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    # typegenes_idx, 
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()

# output = os.path.join(outfigdir, "volcano.pdf")
# powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
querygenes = ['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(3,4,figsize=(4*4,4*3), sharex=True, sharey=True)
for cond_idx, thecond in enumerate(unq_conds):
    ax = axs.flat[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
genes_sig = genes_comm[np.any(np.abs(lfc) > 1, axis=1)]
querygenes = np.intersect1d(gene_modules.annots['tf'], genes_sig) #['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(3,4,figsize=(4*4,4*3), sharex=True, sharey=True)
for cond_idx, thecond in enumerate(unq_conds):
    ax = axs.flat[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()